# Wind speed invertion

In [ ]:
import xsar
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

## read sarwing owi file

download more from https://cyclobs.ifremer.fr/static/sarwing_datarmor/processings/c39e79a/default/reports/shoc_dailyupdate_names/report.html (in the 'safe' column, download files named like `*-owi-xx-*.nc`)

In [ ]:
sarwing_owi_file = xsar.get_test_file('s1a-iw-owi-xx-20210909t130650-20210909t130715-039605-04AE83.nc')
sarwing_ds = xr.open_dataset(sarwing_owi_file)
sarwing_ds = xr.merge([sarwing_ds, xr.open_dataset(sarwing_owi_file, group='owiInversionTables_UV')])
sarwing_ds = sarwing_ds.rename_dims({'owiAzSize': 'atrack', 'owiRaSize': 'xtrack'})
sarwing_ds

## get ancillary wind

Ecmwf wind is stored in owi file in *geographical* (deg N/S) convention. `xsarsea.windspeed` need it in *antenna* convention, as a complex number

So we need to substract the heading to get it relative to azimuth, and add 90° to get it relative to antenna.

In [ ]:
owi_ecmwf_wind = sarwing_ds.owiEcmwfWindSpeed * np.exp(1j* np.deg2rad(sarwing_ds.owiEcmwfWindDirection - sarwing_ds.owiHeading + 90))
sarwing_ds = xr.merge([
    sarwing_ds,
    owi_ecmwf_wind.to_dataset(name='owi_ancillary_wind'),
])


## copol inversion

In [ ]:
windspeed_co = windspeed.invert_from_gmf(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        sarwing_ds.owi_ancillary_wind,
        gmf='cmod5n')


## differences with sarwing

In [ ]:
windspeed_diff = windspeed_co - sarwing_ds.owiWindSpeed_Tab_copol
(
    (hv.Image(windspeed_co).opts(title='xsarsea') + hv.Image(sarwing_ds.owiWindSpeed_Tab_copol).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,32))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## dualpol inversion

In [ ]:
sarwing_luts_subset_path = xsar.get_test_file('sarwing_luts_subset')
windspeed.sarwing_luts.register(sarwing_luts_subset_path)

windspeed_dual = windspeed.invert_from_gmf(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        sarwing_ds.owiNrcs_cross,
        windspeed.nesz_flattening(sarwing_ds.owiNesz_cross, sarwing_ds.owiIncidenceAngle),
        sarwing_ds.owi_ancillary_wind,
        gmf=('cmod5n','cmodms1ahw'))

## differences with sarwing

In [ ]:
windspeed_diff = windspeed_dual - sarwing_ds.owiWindSpeed_Tab_dualpol_2steps
(
    (hv.Image(windspeed_dual).opts(title='xsarsea') + 
     hv.Image(sarwing_ds.owiWindSpeed_Tab_dualpol_2steps).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## Adding your own GMF

A *Geophysical Modeling Function * (GMF) is a function that return a simulated *sigma0* from wind condition and instrument incidence angle.

To register a new GMF with `xsarsea.windspeed`, you have to follow the following rules:

  * parameters are `(incidence, windspeed)` or `(incidence, windspeed, phi)`
    * `incidence` is in degrees
    * `windspeed` is in m/s
    * `phi` is wind direction, in degrees, relative to antenna look (0 is downwind in the antenna direction)
    
  * all parameters must be **float**. numpy array are not allowed. `xsarsea.windspeed` will vectorize the function with numba to allow numpy arrays.
  * returned sigma0 must be **linear**, not dB.

In [ ]:
@windspeed.utils.register_cmod(inc_range=[17., 50.], wspd_range=[3., 80.], phi_range=None)
def dummy_gmf(incidence, speed, phi=None):
    a0 = 0.00013106836021008122
    a1 = -4.530598283705591e-06
    a2 = 4.429277425062766e-08
    b0 = 1.3925444179360706
    b1 = 0.004157838450541205
    b2 = 3.4735809771069953e-05

    a = a0 + a1 * incidence + a2 * incidence ** 2
    b = b0 + b1 * incidence + b2 * incidence ** 2
    sig = a * speed ** b

    return sig


In [ ]:
windspeed_dual = windspeed.invert_from_gmf(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        sarwing_ds.owiNrcs_cross,
        windspeed.nesz_flattening(sarwing_ds.owiNesz_cross, sarwing_ds.owiIncidenceAngle),
        sarwing_ds.owi_ancillary_wind,
        gmf=('cmod5n','dummy_gmf'))

In [ ]:
windspeed_diff = windspeed_dual - sarwing_ds.owiWindSpeed_Tab_dualpol_2steps
(
    (hv.Image(windspeed_dual).opts(title='xsarsea') + 
     hv.Image(sarwing_ds.owiWindSpeed_Tab_dualpol_2steps).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-5,5), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))